In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [4]:
train = pd.read_csv("location_train.csv")
test = pd.read_csv("location_test.csv")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 448 entries, ID to 446
dtypes: int64(448)
memory usage: 13.7 MB


In [6]:
train.head()

,ID,class,1,2,3,4,5,6,7,8,...,437,438,439,440,441,442,443,444,445,446
0,0,11,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,8,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Columns: 447 entries, ID to 446
dtypes: int64(447)
memory usage: 3.4 MB


In [8]:
test.head()

,ID,1,2,3,4,5,6,7,8,9,...,437,438,439,440,441,442,443,444,445,446
0,4000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,4001,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,4002,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4003,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
X_train = train.drop(["ID", "class"], axis=1)
y_train = train["class"]

X_test = test.drop(["ID"], axis=1)

# Model selection

In [10]:
# Grid Search

cv = 10         # number of folds
verbose = 3     # information shown during training

## Multi-layer Perceptron

In [11]:
parameters = {
    "hidden_layer_sizes":[(100,), (200,), (300,), (400,), (500,)],
    "solver":["lbfgs"],
    "alpha":[0.0001, 0.001]}
model = GridSearchCV(MLPClassifier(), parameters, cv=cv, verbose=verbose, scoring="f1_weighted")
model.fit(X_train, y_train)

results = pd.DataFrame(model.cv_results_)
results = results[["param_hidden_layer_sizes", "param_alpha", "mean_test_score", "std_test_score"]]
results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.719 total time=   1.6s
[CV 2/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.784 total time=   1.9s
[CV 3/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.726 total time=   2.0s
[CV 4/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.782 total time=   1.7s
[CV 5/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.756 total time=   1.6s
[CV 6/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.731 total time=   1.6s
[CV 7/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.740 total time=   1.6s
[CV 8/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.719 total time=   1.7s
[CV 9/10] END alpha=0.0001, hidden_layer_sizes=(100,), solver=lbfgs;, score=0.716 total time=   1.6s
[CV 10/10] END alpha=0.0001,

,param_hidden_layer_sizes,param_alpha,mean_test_score,std_test_score
7,"(300,)",0.001,0.763437,0.025275
1,"(200,)",0.0001,0.759047,0.026482
9,"(500,)",0.001,0.757983,0.027545
3,"(400,)",0.0001,0.755528,0.027612
6,"(200,)",0.001,0.752925,0.016239
8,"(400,)",0.001,0.751937,0.022393
2,"(300,)",0.0001,0.749238,0.020725
4,"(500,)",0.0001,0.745341,0.034551
5,"(100,)",0.001,0.742192,0.030567
0,"(100,)",0.0001,0.740270,0.024211


# Final model

In [11]:
best_model = model.best_estimator_
best_model.fit(X_train, y_train)

MLPClassifier(alpha=0.01, hidden_layer_sizes=(400,), solver='lbfgs')

In [12]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = best_model.predict(X_test)

In [13]:
predictions.to_csv("submission.csv", index=False)